# **Tarea 1:** EDA y modelos bayesianos
## **Grupo 5** 
## **Integrantes:** 
 * Diego Irarrazaval
 * Pablo Paredes
 * Tomas Rojas

## Pregunta 1:  Carga y limpieza de datos.
### P1.1

In [1]:
import pandas as pd
import glob as glob
import numpy as np

`files_raw`, `files_estadisticas` y `files_asignacion` son listas que contienen las direcciones donde se encuentran los .csv a leer. 

In [2]:
files_raw = glob.glob('data/raw/**/*.csv', recursive = True)
files_estadisticas = glob.glob('data/estadisticas_upz/*.csv')
files_asignacion = glob.glob('data/asignacion_upz/*.csv')

data_raw = ([pd.read_csv(dir) for dir in files_raw])

### Creación del DataFrame y reporte de archivos furnished

In [3]:
'''
Creamos un data frame 'furnished', el cual tendrá dos columnas
1) 'url' para hacer el merge finalmente y obtener el data frame requerido
2) 'furnished' para contar cuantos datos están en archivos furnished y no en archivos all
'''

data_all = []
data_fur = []

for i in [0,2,4,6,8]:
    
    df1 = pd.read_csv(files_raw[i])
    df2 = pd.read_csv(files_raw[i+1])
    
    data_all.append(df1)
    data_fur.append(df2)
    
df_all = pd.concat(data_all)
df_fur = pd.concat(data_fur)
    
f1 = pd.merge(df_all, df_fur, how='outer', on='url', indicator='furnished')
furnished = f1[['url', 'furnished']].copy()

furnished.drop_duplicates(inplace = True)
furnished.reset_index(drop=True, inplace=True)

# Se reportan si hay datos de archivos furnished que no estén en all 

print('Hay '+ str(len(furnished[furnished['furnished'] == 'right_only'])) + ' datos de archivos furnished que no estan en all')

Hay 4 datos de archivos furnished que no estan en all


In [4]:
'''
Creamos el data frame 'data'
'''

df_aux = pd.concat([df_all, df_fur], ignore_index=True)

data = pd.merge(df_aux, furnished, how='inner', on='url')
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)

# Se elimina la columna 'furnished' y se quitan los duplicados

data.drop('furnished', axis=1, inplace=True)
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)

### P1.2 Limpieza de Columnas 

In [5]:
'''
Limpieza de columnas 'price', 'surface', 'n_rooms', 'n_bath'
'''

# Columna de precio ('price') tipo float

data.price = data['price'].str.replace('.', '')
data.price = data['price'].str.strip('$')
data.price = data['price'].map(float)

# Columna de área ('surface') tipo float

data.surface = data['surface'].replace('m2', '', regex=True)
data.surface = data['surface'].map(float)

In [6]:
# Notamos que en la columna de dormitorios ('n_rooms') existe la opción '5+'
# por lo que dejaremos esta columna como categórica

data.n_rooms.unique()

array(['3', 3.0, '5', '4', '2', '1', nan, 5.0, 4.0, '5+', 2.0, 1.0],
      dtype=object)

In [7]:
# Se crea un diccionario para pasar los datos numéricos de 'n_rooms' a string
# y se efectúa el mapeo

dic = {3.0: '3', 5.0: '5', 4.0:'4', 2.0:'2', 1.0:'1'}

data.n_rooms = data['n_rooms'].replace(dic)

In [8]:
# Como habían datos que solo se diferenciaban en la cantidad de dormitorios
# por el tipo de dato que eran (float o int), puede haberse creado duplicados. Se borran nuevamente los duplicados
# de data

data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)

In [9]:
# Se hace lo mismo con la columnna de cantidad de baños ('n_bath')

data.n_bath.unique()

array(['2', 2.0, '1', '4', '3', nan, 3.0, '5', 4.0, 5.0, '5+', 1.0],
      dtype=object)

In [10]:
dic = {2.0:'2', 3.0:'3', 4.0:'4', 5.0:'5', 1.0:'1'}

data.n_bath.replace(dic)
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)

In [11]:
'''
Separación de columna property_tipe|rent_type|location en tres columnas
con los nombres respectivos

'''

# Renombramos la columna

data.columns = ['PTL', 'price', 'n_rooms', 'n_bath', 'surface', 'details', 'url', 'metrocuadrado_index']

In [12]:
# Creamos las columnas y las llenamos

col = data['PTL'].str.split(', ', expand=True)

meta_col = col[0].str.split(' en ', expand=True)

# Nos aseguramos que hayan solo las siguientes opciones:
# -> 'Casa', 'Apartamento' para property_type
# -> 'Arriendo', 'Venta Y Arriendo' para rent_type

print(meta_col[0].unique())
print(meta_col[1].unique())

['Casa' 'Apartamento']
['Arriendo' 'Venta Y Arriendo']


In [13]:
# Formamos las nuevas columnas 'property_type', 'rent_type', 'location'

data['property_type'] = meta_col[0] 
data['rent_type'] = meta_col[1]
data['location'] = col[1]

# y retiramos la columna PTL

data.drop('PTL', axis=1, inplace=True)

In [14]:
# Finalmente quitamos la ciudad de 'location'

loc = col[1].str.split(' Bogotá', expand=True)
data.location = loc[0]

### P1.3 Precio por metro cuadrado y Cantidad de garages

In [15]:
'''
Agregamos una columna que represente el precio por metro cuadrado 'price_per_m2'

'''

data['price_per_m2'] = data['price']/data['surface']

In [16]:
'''
Obtenemos la cantidad de garajes y lo agregamos como columna también 'cant_garajes'

'''

garajes_list = data.url.str.split('-garajes', expand=True)
garajes_num = garajes_list[0].str.rsplit('-', n=1, expand=True)

# indices que tienen urls con info de la cantidad de garajes
ind = garajes_list[1].index[garajes_list[1].isna() == False]

# generación de nueva columna para después asignarla a la data
garajes_list[2] = np.nan
garajes_list[2].loc[ind] = garajes_num[1].loc[ind]

# agregación de la cantidad de garajes a la data (nan si no hay info)
data['cant_garajes'] = garajes_list[2]

C:\Users\Diego\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### P1.4 Clasificación Tipo de Producto

In [17]:
'''
Creamos una nueva columna 'clasif_prod_type' donde se representará la clasificación
de la vivienda con dígitos del 1 al 8 de acuerdo al enunciado

'''

data['clasif_prod_type'] = np.nan

data.clasif_prod_type.loc[(data.property_type == 'Casa') & (data.surface >= 80) & (data.surface < 120)] = 1
data.clasif_prod_type.loc[(data.property_type == 'Casa') & (data.surface >= 120) & (data.surface < 180)] = 2
data.clasif_prod_type.loc[(data.property_type == 'Casa') & (data.surface >= 180) & (data.surface < 240)] = 3
data.clasif_prod_type.loc[(data.property_type == 'Casa') & (data.surface >= 240) & (data.surface < 360)] = 4
data.clasif_prod_type.loc[(data.property_type == 'Casa') & (data.surface >= 360) & (data.surface < 460)] = 5
data.clasif_prod_type.loc[(data.property_type == 'Apartamento') & (data.surface >= 40) & (data.surface < 60)] = 6
data.clasif_prod_type.loc[(data.property_type == 'Apartamento') & (data.surface >= 60) & (data.surface < 80)] = 7
data.clasif_prod_type.loc[(data.property_type == 'Apartamento') & (data.surface >= 80) & (data.surface < 120)] = 8

### P1.5 Obtención del código UPZ

In [18]:
'''
Se carga el archivo y se guarda en un dataframe data_upz
Luego, se realiza el merge para obtener el código para cada barrio

'''

# Se guarda la base de datos en un data frame
data_upz = pd.read_csv(files_asignacion[0], usecols = ['UPlCodigo', 'pro_location', 'UPlArea'])

# Se deja todo en minuscula para poder hacer el merge correctamente
data_upz.pro_location = data_upz.pro_location.map(str).map(lambda s: s.lower())
data.location = data.location.map(lambda s: s.lower())

In [19]:
#Se realiza el merge y se reportan cuantos datos no fueron asignados con código UPZ

data_merge = pd.merge(data, data_upz, left_on='location', right_on='pro_location', how='left')

print('A ' + str(sum(data_merge.UPlCodigo.isna())) + ' datos no se les puede asignar código UPZ')
print('lo cual es ' + str(sum(data_merge.UPlCodigo.isna())/len(data)*100) + '% de los datos')

A 2059 datos no se les puede asignar código UPZ
lo cual es 11.195693545756075% de los datos


In [20]:
# Creamos la columna 'UPZ' en data y le asignamos el código UPZ obtenido en data_merge

data['UPZ'] = data_merge['UPlCodigo']
data['UPZ_area'] = data_merge['UPlArea']

### P1.6 Fusión de datos con código UPZ

In [21]:
'''
Se cargan los datos en distintos data frames, para después hacerles merge con data
Luego, se crea una columna de densidad poblacional para cada código UTZ

'''

# Se cargan los datos en data frames respectivamente

data_pobl = pd.read_csv(files_estadisticas[0])
data_inseg = pd.read_csv(files_estadisticas[1])
data_verde = pd.read_csv(files_estadisticas[2])

In [22]:
# Se quitan las columnas innecesarias de data_pobl y data_inseg

data_pobl.drop(['Unnamed: 0', 'nomupz'], axis=1, inplace=True)
data_inseg.drop(['Unnamed: 0', 'UPlNombre2'], axis=1, inplace=True)

In [23]:
# En data_verde se tiene que al código UPZ viene solo el número
# por lo tanto, hay que transformarlo al formato UPZ + número para poder hacer el merge correctamente

col = data_verde.cod_upz.map(int).map(str)
col_upz = 'UPZ' + col

data_verde.cod_upz = col_upz

# Se eliminan las columnas innecesarias
data_verde.drop(['Unnamed: 0', 'upz'], axis=1, inplace=True)

In [24]:
# Se realiza el merge, eliminando después las columnas innecesarias

data = pd.merge(data, data_pobl, left_on='UPZ', right_on='upz', how='left')
data.drop('upz', axis=1, inplace=True)
data = pd.merge(data, data_inseg, left_on='UPZ', right_on='UPlCodigo', how='left')
data.drop('UPlCodigo', axis=1, inplace=True)
data = pd.merge(data, data_verde, left_on='UPZ', right_on='cod_upz', how='left')
data.drop('cod_upz', axis=1, inplace=True)

In [25]:
# Finalmente, se crea la columna de densidad de población para cada código UTZ ('UTZ_density')

data['UTZ_density'] = data.personas/data.UPZ_area

## P2. EDA
### P2.1 Creacion de `estilo()`

In [26]:
import seaborn as sb
#se crea diccionario que dará los valores a setear por defecto en el notebook
custom = {
    "font.size": 12,
    "axes.labelsize": 18,
    "axes.titlesize": 18,
    "xtick.labelsize": 18,
    "ytick.labelsize": 18,
    "legend.fontsize": 20,
    "axes.linewidth": 1.25,
    "grid.linewidth": 1,
    "lines.linewidth": 1.5,
    "lines.markersize": 6,
    "patch.linewidth": 1,
    "xtick.major.width": 1.25,
    "ytick.major.width": 1.25,
    "xtick.minor.width": 1,
    "ytick.minor.width": 1,
    "xtick.major.size": 6,
    "ytick.major.size": 6,
    "xtick.minor.size": 4,
    "ytick.minor.size": 4,
    'figure.figsize':(10.,8.),
    "figure.facecolor": "white",
    "axes.labelcolor": ".15",
    "xtick.direction": "out",
    "ytick.direction": "out",
    "xtick.color": ".15",
    "ytick.color": ".15",
    "axes.axisbelow": True,
    "grid.linestyle": "--",
    "text.color": ".1",
    "patch.force_edgecolor": True,
    "image.cmap": "rocket",
    "xtick.top": False,
    "ytick.right": False,
         }

#En las siguiente línea se implementa el diccionario personalizado como default para este notebook en seaborn
sb.set(rc=custom)

#se escoge una de las paletas que vienen con seaborn 
#(distinta a la que se usa por defecto) para el resto de notebook.
sb.set_palette('Set2')

# P3

In [28]:
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
from scipy.stats import multivariate_normal

def convergence(old, new, tol):
    test = np.abs(new - old) / old
    if test <= tol:
        return True
    else:
        return False


def calculate_eigs(beta, X):
    M = beta * np.matmul(X.T, X)
    return np.linalg.eig(M)[0]

def get_S_N(alpha, beta, X):
    a = beta * np.matmul(X.T, X)
    b = alpha * np.identity(a.shape[0])
    to_return = a + b
    return np.linalg.inv(to_return)


def get_m_N(beta, S_N, X, y):
    return beta * S_N.dot(X.T.dot(y))


def get_alpha(gamma, m_N):
    return gamma / np.matmul(m_N.T, m_N)


def get_beta(N, gamma, m_N, X, y):
    """
    Funcion que nos da beta para una iteracion dada
    :param N:
    :param gamma: parametro gamma
    :param m_N: matriz
    :param X: matrix con los vectores fila x_i uno sobre el otro
    :param y: vector con las salidas
    :return: devuelve un valor para beta
    """
    m_N_X = np.matmul(m_N.T, X.T)
    suma = y - m_N_X
    suma = suma ** 2
    return (suma.sum(axis=0) / (N - gamma)) ** -1


def get_gamma(alpha, beta, X):
    eig = calculate_eigs(beta, X)
    terms = np.array([i / (alpha + i) for i in eig])
    return terms.sum()


class RegresionBayesianaEmpirica(BaseEstimator, RegressorMixin):
    def __init__(self, alpha_0, beta_0, tol=1e-5, maxiter=200):
        """
        :param alpha_0: initial guess for alpha
        :param beta_0: initial guess for beta
        :param tol: tolerance
        :param maxiter: maximum iterations
        """
        # valores iniciales
        self.alpha_0 = alpha_0
        self.beta_0 = beta_0

        # propiedades de convergencia
        self.tol = tol
        self.maxiter = maxiter

        self.alpha = None
        self.beta = None
        self.gamma = None

        self.posteriori = None



    def get_posteriori(self, X, y, alpha, beta):
        """
        Esta func tiene que darnos la dist posterior
        :param X:
        :param y:
        :param alpha:
        :param beta:
        :return:
        """
        S_N = get_S_N(alpha, beta, X)
        m_N = get_m_N(beta, S_N, X, y)
        mu = m_N.T.dot(X.T)
        var = lambda x: (1 / beta) + X.dot(S_N.dot(x))
        self.posteriori = lambda x: multivariate_normal(mu, var(x))
        return


    def fit(self, X, y):
        """
        Fitea los datos en la dist
        :param X: Datos con vectores columna uno al lado del otro
        :param y: vector vertical con los resultados
        :return: None
        """

        N = len(y)
        c_alpha = self.alpha_0
        c_beta = self.beta_0
        c_gamma = get_gamma(c_alpha, c_beta, X)
        c_S_N = get_S_N(c_alpha, c_beta, X)
        c_m_N = get_m_N(c_beta, c_S_N, X, y)
        i = 0

        while i < self.maxiter:
            i += 1
            if i % 25 == 0:
                print('{}/{}'.format(i, self.maxiter))

            n_alpha = get_alpha(c_gamma, c_m_N)
            n_beta = get_beta(N, c_gamma, c_m_N, X, y)
            n_gamma = get_gamma(c_alpha, c_beta, X)
            c_S_N = get_S_N(n_alpha, n_beta, X)
            c_m_N = get_m_N(n_beta, c_S_N, X, y)

            # convergence
            for_conv_1 = convergence(c_alpha, n_alpha, self.tol)
            for_conv_2 = convergence(c_beta, n_beta, self.tol)
            for_conv_3 = convergence(c_gamma, n_gamma, self.tol)

            # break condition
            if for_conv_1 and for_conv_2 and for_conv_3:
                break

            c_alpha = n_alpha.copy()
            c_beta = n_beta.copy()
            c_gamma = n_gamma.copy()

        self.alpha = n_alpha
        self.beta = n_beta
        self.gamma = n_gamma
        self.get_posteriori(X, y, self.alpha, self.beta)

        return

    def predict(self, X, return_std=False):
        distribution = self.posteriori(X)
        return distribution(X.T)

# Prueba del modelo con otro dataset:
https://www.kaggle.com/aungpyaeap/fish-market/data


In [38]:
fish = pd.read_csv('Fish.csv')

In [39]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Species  159 non-null    object 
 1   Weight   159 non-null    float64
 2   Length1  159 non-null    float64
 3   Length2  159 non-null    float64
 4   Length3  159 non-null    float64
 5   Height   159 non-null    float64
 6   Width    159 non-null    float64
dtypes: float64(6), object(1)
memory usage: 8.8+ KB


In [42]:
from sklearn.model_selection import train_test_split
X_cols = [col for col in fish.columns if col not in ['Weight', 'Species']]
print('X_cols = ', X_cols)
X = fish[X_cols]
y = fish['Weight']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_cols =  ['Length1', 'Length2', 'Length3', 'Height', 'Width']


In [49]:
X_train

,Length1,Length2,Length3,Height,Width
69,21.2,23.0,25.8,10.3458,3.6636
127,41.1,44.0,46.6,12.4888,7.5958
27,32.7,36.0,41.5,16.5170,5.8515
150,10.8,11.3,12.6,1.9782,1.2852
124,39.8,43.0,45.2,11.9328,7.2772
...,...,...,...,...,...
71,24.0,26.0,29.0,11.3680,4.2340
106,25.9,28.0,29.4,7.8204,4.2042
14,29.4,32.0,37.2,14.9544,5.1708
92,20.5,22.5,24.0,6.7920,3.6240


In [50]:
X_train.T

,69,127,27,150,124,41,4,134,32,140,...,87,74,121,158,20,71,106,14,92,102
Length1,21.2000,41.1000,32.7000,10.8000,39.8000,19.1000,26.500,40.0000,36.2000,48.3000,...,20.000,13.800,37.0000,13.8000,31.3000,24.000,25.9000,29.4000,20.500,25.2000
Length2,23.0000,44.0000,36.0000,11.3000,43.0000,20.8000,29.000,42.5000,39.5000,51.7000,...,22.000,15.000,40.0000,15.0000,34.0000,26.000,28.0000,32.0000,22.500,27.3000
Length3,25.8000,46.6000,41.5000,12.6000,45.2000,23.1000,34.000,45.5000,45.3000,55.1000,...,23.500,16.000,42.4000,16.2000,39.5000,29.000,29.4000,37.2000,24.000,28.7000
Height,10.3458,12.4888,16.5170,1.9782,11.9328,6.1677,12.444,7.2800,18.7542,8.9262,...,5.640,3.824,12.3808,2.9322,15.1285,11.368,7.8204,14.9544,6.792,8.3230
Width,3.6636,7.5958,5.8515,1.2852,7.2772,3.3957,5.134,4.3225,6.7497,6.1712,...,3.525,2.432,7.4624,1.8792,5.5695,4.234,4.2042,5.1708,3.624,5.1373


In [48]:
rg = RegresionBayesianaEmpirica(1e-5,1e-5)
rg.fit(X_train.T,y_train)

ValueError: Shape of passed values is (106, 106), indices imply (106, 5)